In [1]:
from datasets import load_dataset
#from googletrans import Translator
from tqdm import tqdm
from llm.GepetoBase import ChatGPT
import pandas as pd

In [3]:
# Função para traduzir o texto de inglês para português
gpt = ChatGPT('../key.txt') 

def traduzir_texto(chat_gpt, prompt):
    
    chat_gpt.add_message(role="user", prompt=prompt)

    # fazendo inferencia
    resposta, _ = chat_gpt.get_completion()

    chat_gpt.reset_messages(maintain_context=False)
    
    return resposta

In [4]:
# estrutura para armazenar os dados processados. Cada elemento da lista é uma lista com 2 elementos: pergunta e resposta.
dados = []

# Pré-processando o dataset MedicationQA

In [5]:
medicationqa = load_dataset('truehealth/medicationqa')['train']

In [6]:
medicationqa[0]

{'Question': 'how does rivatigmine and otc sleep medicine interact',
 'Focus (Drug)': 'rivastigmine',
 'Question Type': 'Interaction',
 'Answer': "tell your doctor and pharmacist what prescription and nonprescription medications, vitamins, nutritional supplements, and herbal products you are taking or plan to take. Be sure to mention any of the following: antihistamines; aspirin and other nonsteroidal anti-inflammatory medications (NSAIDs) such as ibuprofen (Advil, Motrin) and naproxen (Aleve, Naprosyn); bethanechol (Duvoid, Urecholine); ipratropium (Atrovent, in Combivent, DuoNeb); and medications for Alzheimer's disease, glaucoma, irritable bowel disease, motion sickness, ulcers, or urinary problems. Your doctor may need to change the doses of your medications or monitor you carefully for side effects.",
 'Section Title': 'What special precautions should I follow?',
 'URL': 'https://medlineplus.gov/druginfo/meds/a602009.html'}

In [10]:
for index in tqdm(range(len(medicationqa))):
    
    prompt = f"Você irá receber uma pergunta e uma resposta, sobre medicamentos, em inglês. Você deve traduzir a pergunta e a resposta para o português. Para isso, fique atento a possíveis abreviações, expressões populares e polissemias, e traduza da melhor forma possível para o português, removendo possíveis ruídos do texto também. A pergunta está marcada com a tag <pergunta> e a resposta com a tag <resposta>. Há APENAS UMA PERGUNTA E UMA RESPOSTA.\nRetorne a tradução da pergunta e da resposta em um json, de modo que a pergunta e a resposta traduzidas estejam em chaves 'pergunta' e 'resposta'. Segue um exemplo de retorno: '{{'pergunta':'pergunta traduzida', 'resposta':'resposta traduzida'}}'.\n\n <pergunta> {medicationqa[index]['Question']} <pergunta> \n\n <resposta> {medicationqa[index]['Answer']} <resposta> \n."
    
    try:
        
        resposta = traduzir_texto(gpt, prompt).replace('\n', ' ')
    
        if resposta[-1] != "}":
            resposta = resposta[:-1]
            
        resp = eval(resposta)
        
        dados.append([resp['pergunta'], resp['resposta']])
        
    except SyntaxError:
        continue
    
    except:
        gpt.reset_messages(maintain_context=False)
        resposta = traduzir_texto(gpt, prompt[:int(len(prompt)*0.75)]).replace('\n', ' ')
    
        if resposta[-1] != "}":
            resposta = resposta[:-1]
        
        try:
            resp = eval(resposta)
            
            dados.append([resp['pergunta'], resp['resposta']])
            
        except:
            continue

100%|██████████| 562/562 [56:53<00:00,  6.07s/it]   


# Pré-processando o dataset LiveQA

In [26]:
liveqa = load_dataset('truehealth/liveqa')['train']

In [14]:
liveqa[0]

{'questionid': 'Q1',
 'subject': None,
 'message': 'Literature on Cardiac amyloidosis.  Please let me know where I can get literature on Cardiac amyloidosis.  My uncle died yesterday from this disorder.  Since this is such a rare disorder, and to honor his memory, I would like to distribute literature at his funeral service.  I am a retired NIH employee, so I am familiar with the campus in case you have literature at NIH that I can come and pick up.  Thank you ',
 'focus': 'cardiac amyloidosis',
 'type': 'information',
 'answerid': 'Q1-S1-A1',
 'pairid': '1',
 'answer': 'Cardiac amyloidosis is a disorder caused by deposits of an abnormal protein (amyloid) in the heart tissue. These deposits make it hard for the heart to work properly.'}

In [42]:
for index in tqdm(range(len(liveqa))):
    prompt = f"Você irá receber uma pergunta e uma resposta, sobre dúvidas de consumidores, enviadas para especialistas médicos, em inglês. Você deve traduzir a pergunta e a resposta para o português. Para isso, fique atento a possíveis abreviações, expressões populares e polissemias, e traduza da melhor forma possível para o português, removendo possíveis ruídos do texto também. A pergunta está marcada com a tag <pergunta> e a resposta com a tag <resposta>.\n Retorne a tradução da pergunta e da resposta em um json, de modo que a pergunta e a resposta traduzidas estejam em chaves 'pergunta' e 'resposta'. Segue um exemplo de retorno: '{{'pergunta':'pergunta traduzida', 'resposta':'resposta traduzida'}}'.\n\n <pergunta> {liveqa[index]['message']} <pergunta> \n\n <resposta> {liveqa[index]['answer']} <resposta> \n."
    
    
    try:
        
        resposta = traduzir_texto(gpt, prompt).replace('\n', ' ')
    
        if resposta[-1] != "}":
            resposta = resposta[:-1]
            
        resp = eval(resposta)
        
        dados.append([resp['pergunta'], resp['resposta']])
        
    except SyntaxError:
        continue
    
    except:
        gpt.reset_messages(maintain_context=False)
        resposta = traduzir_texto(gpt, prompt[:int(len(prompt)*0.75)]).replace('\n', ' ')
    
        if resposta[-1] != "}":
            resposta = resposta[:-1]
        
        try:
            resp = eval(resposta)
            
            dados.append([resp['pergunta'], resp['resposta']])
            
        except:
            continue

100%|██████████| 339/339 [1:11:45<00:00, 12.70s/it]   


# Pré-processando o dataset MedMCQA

In [61]:
medmcqa = load_dataset('medmcqa')['train'].select(list(range(10000)))

In [ ]:
for index in tqdm(range(len(medmcqa))):
    pergunta = medmcqa[index]['question']+"\n"
    pergunta+=f"A){medmcqa[index]['opa']}\n"
    pergunta+=f"B){medmcqa[index]['opb']}\n"
    pergunta+=f"C){medmcqa[index]['opc']}\n"
    pergunta+=f"D){medmcqa[index]['opd']}\n"
    
    correta = ['A','B', 'C', 'D']
    
    try:
        resposta = f"Alternativa {correta[medmcqa[index]['cop']]}. {medmcqa[index]['exp']}."
    except:
        resposta = f"Alternativa {correta[medmcqa[index]['cop']]}."
    
    prompt = f"Você irá receber uma questão de múltipla escolha, em inglês, de modo que a você será passado o enunciado, junto com as alternativas, demarcadas por 'A)', 'B)', 'C)' e 'D)'. Além disso, você também receberá a resposta dessa questão, que pode conter apenas a letra correta, mas também a letra junto com uma explicação. Você deve traduzir a pergunta e a resposta para o português. Ao traduzir a pergunta, mantenha as alternativas 'A)', 'B)', 'C)' e 'D)', com suas respectivas traduções, não fuja desse formato. Também, fique atento a possíveis abreviações, expressões populares e polissemias, e traduza da melhor forma possível para o português, removendo possíveis ruídos do texto também. A pergunta/questão está marcada com a tag <pergunta> e a resposta com a tag <resposta>.\n Retorne a tradução da pergunta e da resposta em um json, de modo que a pergunta e a resposta traduzidas estejam em chaves 'pergunta' e 'resposta'. Além disso, crie chaves para cada alternativa possível. Segue um exemplo, a ser seguido por você, de retorno: '{{'pergunta':'enunciado da questao', 'A': 'letra A', 'B': 'letra B', 'C': 'letra C', 'D': 'letra D', 'resposta':'resposta traduzida'}}'.\n\n <pergunta> {pergunta} <pergunta> \n\n <resposta> {resposta} <resposta> \n."
    
    
    try:
        
        resposta = traduzir_texto(gpt, prompt).replace('\n', ' ')
    
        if resposta[-1] != "}":
            resposta = resposta[:-1]
            
        resp = eval(resposta)
        p = resp['pergunta']+"\n"
        p+=f"A){resp['A']}\n"
        p+=f"B){resp['B']}\n"
        p+=f"C){resp['C']}\n"
        p+=f"D){resp['D']}\n"
        
        dados.append([p, resp['resposta']])
        
    except SyntaxError:
        continue
    
    except:
        gpt.reset_messages(maintain_context=False)
        resposta = traduzir_texto(gpt, prompt[:int(len(prompt)*0.75)]).replace('\n', ' ')
    
        if resposta[-1] != "}":
            resposta = resposta[:-1]
        
        try:
            resp = eval(resposta)
            p = resp['pergunta']+"\n"
            p+=f"A){resp['A']}\n"
            p+=f"B){resp['B']}\n"
            p+=f"C){resp['C']}\n"
            p+=f"D){resp['D']}\n"
            
            dados.append([p, resp['resposta']])
            
        except:
            continue


 19%|█▉        | 1896/9977 [3:39:01<13:27:41,  6.00s/it]  

# Pré-processando o dataset MedQA(USMLE)

In [75]:
medqa = load_dataset('GBaker/MedQA-USMLE-4-options')['train']

Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

In [86]:
for index in tqdm(range(len(medqa))):
    
    pergunta = medqa[index]['question']+"\n"
    pergunta+=f"A){medqa[index]['options']['A']}\n"
    pergunta+=f"B){medqa[index]['options']['B']}\n"
    pergunta+=f"C){medqa[index]['options']['C']}\n"
    pergunta+=f"D){medqa[index]['options']['D']}\n"
    
    try:
        resposta = f"Alternativa {medqa[index]['answer_idx']}. {medqa[index]['answer']}."
        
    except:
        resposta = f"Alternativa {medqa[index]['answer_idx']}"
        
    
    prompt = f"Você irá receber uma questão de múltipla escolha, em inglês, de modo que a você será passado o enunciado, junto com as alternativas, demarcadas por 'A)', 'B)', 'C)' e 'D)'. Além disso, você também receberá a resposta dessa questão, que pode conter apenas a letra correta, mas também a letra junto com uma explicação. Você deve traduzir a pergunta e a resposta para o português. Ao traduzir a pergunta, mantenha as alternativas 'A)', 'B)', 'C)' e 'D)', com suas respectivas traduções, não fuja desse formato. Também, fique atento a possíveis abreviações, expressões populares e polissemias, e traduza da melhor forma possível para o português, removendo possíveis ruídos do texto também. A pergunta/questão está marcada com a tag <pergunta> e a resposta com a tag <resposta>.\n Retorne a tradução da pergunta e da resposta em um json, de modo que a pergunta e a resposta traduzidas estejam em chaves 'pergunta' e 'resposta'. Além disso, crie chaves para cada alternativa possível. Segue um exemplo, a ser seguido por você, de retorno: '{{'pergunta':'enunciado da questao', 'A': 'letra A', 'B': 'letra B', 'C': 'letra C', 'D': 'letra D', 'resposta':'resposta traduzida'}}'.\n\n <pergunta> {pergunta} <pergunta> \n\n <resposta> {resposta} <resposta> \n."
    
    
    try:
        
        resposta = traduzir_texto(gpt, prompt).replace('\n', ' ')
    
        if resposta[-1] != "}":
            resposta = resposta[:-1]
            
        resp = eval(resposta)
        p = resp['pergunta']+"\n"
        p+=f"A){resp['A']}\n"
        p+=f"B){resp['B']}\n"
        p+=f"C){resp['C']}\n"
        p+=f"D){resp['D']}\n"
        
        dados.append([p, resp['resposta']])
        
    except SyntaxError:
        continue
    
    except:
        gpt.reset_messages(maintain_context=False)
        resposta = traduzir_texto(gpt, prompt[:int(len(prompt)*0.75)]).replace('\n', ' ')
    
        if resposta[-1] != "}":
            resposta = resposta[:-1]
        
        try:
            resp = eval(resposta)
            p = resp['pergunta']+"\n"
            p+=f"A){resp['A']}\n"
            p+=f"B){resp['B']}\n"
            p+=f"C){resp['C']}\n"
            p+=f"D){resp['D']}\n"
            
            dados.append([p, resp['resposta']])
            
        except:
            continue

100%|██████████| 1974/1974 [1:24:37<00:00,  2.57s/it]


# Reunindo os dados captados em um DataFrame

In [22]:
import pandas as pd

In [12]:
df = pd.DataFrame(dados, columns=['instruction', 'output'])

In [32]:
df

,instruction,output
0,Como o rivastigmina e o medicamento para dormi...,Informe ao seu médico e farmacêutico quais med...
1,Como o valium afeta o cérebro?,O diazepam é uma benzodiazepina que exerce efe...
2,O que é morfina?,A morfina é um medicamento para dor da família...
3,Quais são os miligramas para oxycodone?,... 10 mg ... 20 mg ... 40 mg ... 80 mg ...
4,81% de aspirina contém resina e goma-laca nela?,Ingredientes inativos Nome do ingrediente
...,...,...
21122,Um homem de 60 anos se apresenta ao departamen...,Alternativa B. Administrar desmopressina.
21123,Um homem de 45 anos com uma história de 15 ano...,Alternativa B. Razão de microalbumina para cre...
21124,Após receber um resultado positivo no teste de...,Alternativa B. Acetaminofeno.
21125,Um homem de 25 anos vai ao consultório por cau...,Alternativa C. Oxidase de ácido homogentísico.


In [33]:
df.shape

(21127, 2)

In [34]:
df.to_csv('../dados/treino/drbyte_ptbr_treino.csv', index=False)